In [2]:
# Import Libraries
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from ta.others import daily_return,cumulative_return
from plotly.offline import init_notebook_mode
import cufflinks as cf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import requests
from bs4 import BeautifulSoup
import warnings
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
%matplotlib inline
init_notebook_mode(connected=True)
cf.go_offline()

In [3]:
# Check if folder input exists

# Define the folder name
folder_name = '../input'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../input' folder already exists.


In [4]:
# Check if folder tickers exists

# Define the folder name
folder_name = '../tickers'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../tickers' folder already exists.


In [5]:
# Webscrapping Wikipedia table to get the IBEX 35 Growth components
page = requests.get("https://es.wikipedia.org/wiki/IBEX_Small_Cap")
soup = BeautifulSoup(page.text, 'html.parser') 
table = soup.find('table',class_="wikitable sortable")

In [6]:
# Convert html table to dataframe
wiki = pd.read_html(str(table))
wiki = pd.concat(wiki)
wiki.head(40)

,Ticker,Empresa,Sede,Sector[3]​,ISIN
0,AZK,Azkoyen,Peralta,Ingeniería y otros,ES0112458312
1,ADX,Audax Renovables,Badalona,Energías renovables,ES0136463017
2,AEDAS,AEDAS Homes,Madrid,Inmobiliarias y otros,ES0105287009
3,AI,Airtificial,Madrid,Ingeniería y otros,ES0152768612
4,AMP,Amper,Pozuelo de Alarcón,Electrónica y software,ES0109260531
5,BKY,Berkeley Energía,Madrid,"Mineral, metales y transformación",AU000000BKY0
6,DIA,DIA,Las Rozas,Comercio,ES0126775032
7,ECR,Ercros,Barcelona,Industria química,ES0125140A14
8,GSJ,San José,Pontevedra,Construcción,ES0180918015
9,LRE,Lar España,Madrid,SOCIMI,ES0105015012


In [7]:
# Add .MC to all tickers
wiki['Ticker'] = wiki['Ticker'].apply(lambda x: x + '.MC')
wiki

,Ticker,Empresa,Sede,Sector[3]​,ISIN
0,AZK.MC,Azkoyen,Peralta,Ingeniería y otros,ES0112458312
1,ADX.MC,Audax Renovables,Badalona,Energías renovables,ES0136463017
2,AEDAS.MC,AEDAS Homes,Madrid,Inmobiliarias y otros,ES0105287009
3,AI.MC,Airtificial,Madrid,Ingeniería y otros,ES0152768612
4,AMP.MC,Amper,Pozuelo de Alarcón,Electrónica y software,ES0109260531
5,BKY.MC,Berkeley Energía,Madrid,"Mineral, metales y transformación",AU000000BKY0
6,DIA.MC,DIA,Las Rozas,Comercio,ES0126775032
7,ECR.MC,Ercros,Barcelona,Industria química,ES0125140A14
8,GSJ.MC,San José,Pontevedra,Construcción,ES0180918015
9,LRE.MC,Lar España,Madrid,SOCIMI,ES0105015012


In [8]:
wiki.rename(columns={'Empresa':'Name','Sector[3]\u200b':'Sector'}, inplace=True)

In [9]:
# Get the data for the stock index
index_list = wiki['Ticker'].tolist()

In [10]:
# Save all the historical data
for stock in tqdm(index_list):
    data = yf.download(stock, progress=False,multi_level_index=False,actions=True,auto_adjust=False)
    data.to_csv(f"../input/{stock}.csv",index=True)

  0%|          | 0/29 [00:00<?, ?it/s]

In [11]:
# Get the name of the notebook
notebook_name = os.path.basename(globals()['__vsc_ipynb_file__'])
notebook_name = notebook_name.split('-')[0]

In [12]:
# Save all the tickers data
wiki[['Name','Sector','Ticker']].to_csv(f"../tickers/{notebook_name}.csv",index=True)

In [13]:
# Configuration of different parameters of the notebook
ticker = 'ADX.MC'
year = '2025'

In [14]:
# Check DataFrame
stock_ticker = pd.read_csv(f"../input/{ticker}.csv",index_col="Date",parse_dates=True)
stock_ticker.head(10)

,Adj Close,Close,Dividends,High,Low,Open,Stock Splits,Volume
Date,,,,,,,,
2007-04-24,9.147391,9.60,0.0,9.60,9.45,9.60,0.0,15000
2007-04-25,8.861535,9.30,0.0,9.30,9.30,9.30,0.0,12593
2007-04-26,8.861535,9.30,0.0,9.30,9.30,9.30,0.0,18185
2007-04-27,9.147391,9.60,0.0,9.60,9.50,9.60,0.0,9399
2007-04-30,10.062132,10.56,0.0,10.56,10.20,10.56,0.0,95227
2007-05-02,11.338954,11.90,0.0,12.24,11.00,11.80,0.0,0
2007-05-03,9.290319,9.75,0.0,11.95,9.51,11.78,0.0,0
2007-05-04,7.718112,8.10,0.0,9.10,8.05,8.91,0.0,0
2007-05-07,7.870569,8.26,0.0,8.49,8.01,8.01,0.0,0


In [15]:
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,vertical_spacing=0.01,specs=[[{'rowspan':3,'colspan':1}],[None],[None],[{'rowspan':1,'colspan':1}]])

# Graph (1,1)
fig.add_trace(go.Scatter(x=stock_ticker.index,y=stock_ticker['Close'],mode="lines",name=f"{ticker}"),row=1, col=1)
# Update xaxis properties
fig.update_yaxes(title_text="Price", row=1, col=1)

# Graph (4,1)
fig.add_trace(go.Scatter(x=stock_ticker.index,y=stock_ticker['Volume'],mode="lines",name='Volume'),row=4, col=1)
fig.update_yaxes(title_text="Volume", row=4, col=1)

fig.update_layout(height=800, width=1300,showlegend=False,title=f"{ticker}")

fig.show()

In [16]:
# Add Technical Analysis Indicators

# Modified Moving Average 20
stock_ticker['MMA20'] = stock_ticker['Adj Close'].loc[year].rolling(20).mean() #Adj Close 20 MA

# Initialize Bollinger Bands Indicator
indicator_bb = BollingerBands(close=stock_ticker["Adj Close"].loc[year], window=20, window_dev=2)

# Bollinger Bands
stock_ticker['BB_Upper'] = indicator_bb.bollinger_hband()
stock_ticker['BB_Lower'] = indicator_bb.bollinger_lband()

# Initialize RSI Indicator
indicator_rsi = RSIIndicator(close=stock_ticker["Adj Close"].loc[year], window=14)

# RSI
stock_ticker['RSI'] = indicator_rsi.rsi()

# Daily Return
stock_ticker['Daily_Return'] = daily_return(stock_ticker["Adj Close"].loc[year])

In [17]:
# Plot the adjusted close price
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,vertical_spacing=0.01,specs=[[{'rowspan':2,'colspan':1}],[None],[{'rowspan':1,'colspan':1}],[{'rowspan':1,'colspan':1}]])

# Graph (1,1)
fig.add_trace(go.Scatter(x=stock_ticker['Adj Close'].loc[year].index,y=stock_ticker['Adj Close'].loc[year],mode="lines",name=f'{ticker}'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['BB_Lower'].loc[year].index,y=stock_ticker['BB_Lower'].loc[year],mode="lines",name='BB_Lower'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['BB_Upper'].loc[year].index,y=stock_ticker['BB_Upper'].loc[year],mode="lines",name='BB_Upper'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['MMA20'].loc[year].index,y=stock_ticker['MMA20'].loc[year],mode="lines",name='MMA20'),row=1,col=1)
# Update xaxis properties
fig.update_yaxes(title_text="Price", row=1, col=1)

# Graph (3,1)
fig.add_trace(go.Scatter(x=stock_ticker['Volume'].loc[year].index,y=stock_ticker['Volume'].loc[year],mode="lines",name='Volume'),row=3, col=1)
fig.add_trace(go.Scatter(x=stock_ticker['Volume'].loc[year].index,y=stock_ticker['Volume'].loc[year].rolling(20).mean(),mode="lines",name='MMA20'),row=3,col=1)
fig.update_yaxes(title_text="Volume", row=3, col=1)

# Graph (4,1)
fig.add_trace(go.Scatter(x=stock_ticker['RSI'].loc[year].index,y=stock_ticker['RSI'].loc[year],mode="lines",name='RSI'),row=4, col=1)
fig.add_hline(y=30, line_width=1, line_dash="dash", line_color="green",row=4,col=1)
fig.add_hline(y=70, line_width=1, line_dash="dash", line_color="red",row=4,col=1)
fig.update_yaxes(title_text="RSI", row=4, col=1)

fig.update_layout(height=800, width=1300,showlegend=False,title=f"{ticker} {year}")

fig.show()

In [18]:
# Plotly
fig = make_subplots()

# Graph (1,1)
# Loop all stock files and get cummulative return for year
for stock in tqdm(index_list):
    f = os.path.join("../input", stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    fig.add_trace(go.Scatter(x=df['Cummulative_Return'].loc[year].index,y=df['Cummulative_Return'].loc[year],mode="lines",name=stock.split('.')[0]),row=1,col=1)

# Update xaxis properties
fig.update_yaxes(title_text="Return", row=1, col=1)

fig.update_layout(height=800, width=1300,showlegend=True,title=f"Cummulative Returns {ticker} for {year}")

fig.show()

  0%|          | 0/29 [00:00<?, ?it/s]

In [19]:
# Create also table of cummulative returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input", stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Cummulative_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Cummulative_Return'])
cum = wiki[['Ticker','Name']].merge(cum,on='Ticker')
cum.sort_values(by=['Cummulative_Return'],ignore_index=True, ascending=False).head(30)

  0%|          | 0/29 [00:00<?, ?it/s]

,Ticker,Name,Date,Cummulative_Return
0,DIA.MC,DIA,2025-03-14,114279.084321
1,ORY.MC,Oryzon Genomics,2025-03-14,91.078558
2,PRS.MC,Prisa,2025-03-14,28.052808
3,BKY.MC,Berkeley Energía,2025-03-14,27.860693
4,AI.MC,Airtificial,2025-03-14,27.366259
5,TRG.MC,Tubos Reunidos,2025-03-14,26.129669
6,NEA.MC,Nicolás Correa,2025-03-14,24.415587
7,REN.MC,Renta Corporación,2025-03-14,23.030295
8,AMP.MC,Amper,2025-03-14,22.448973
9,CASH.MC,Prosegur Cash,2025-03-14,22.323054


In [20]:
# Create also table of daily returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input",stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Daily_Return'] = daily_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Daily_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Daily_Return'])
cum = wiki[['Ticker','Name']].merge(cum,on='Ticker')
cum.sort_values(by=['Daily_Return'],ignore_index=True, ascending=False).head(40)

  0%|          | 0/29 [00:00<?, ?it/s]

,Ticker,Name,Date,Daily_Return
0,MDF.MC,Duro Felguera,2025-03-14,15.257342
1,TRG.MC,Tubos Reunidos,2025-03-14,6.291399
2,AEDAS.MC,AEDAS Homes,2025-03-14,2.321427
3,REN.MC,Renta Corporación,2025-03-14,2.266996
4,AMP.MC,Amper,2025-03-14,1.982994
5,NEA.MC,Nicolás Correa,2025-03-14,1.914897
6,DOM.MC,Global Dominion Access,2025-03-14,1.798559
7,HOME.MC,Neinor Homes,2025-03-14,1.639346
8,ADX.MC,Audax Renovables,2025-03-14,1.398608
9,ORY.MC,Oryzon Genomics,2025-03-14,1.234563
